In [172]:
import pandas as pd
import numpy as np
data='../../data/kundaje/bulk/croo/'

In [89]:
days=['D0','D2','D4','D6']
peak='/peak/overlap_reproducibility/overlap.optimal_peak.narrowPeak.gz'

In [135]:
dfs={}
for day in days[0:2]:
    df=pd.read_table(data+day+peak,header=None)
    dfs[day]=df

In [151]:
df0=dfs['D0'][[0,1,2]].drop_duplicates()

In [148]:
#ps=(df[0]+':'+df[1].astype(str)+'_'+df[2].astype(str)).values

In [152]:
df021=df0[df0[0]=='chr21']

In [259]:
ps021=(df021[0]+':'+df021[1].astype(str)+'_'+df021[2].astype(str)).values

In [154]:
len(ps021)

1602

In [155]:
df2=dfs['D2'][[0,1,2]].drop_duplicates()

In [156]:
df221=df2[df2[0]=='chr21']

In [231]:
df221['r']=df221.apply(lambda x:set(range(x[1],x[2])),1)

/home/moqri/anaconda3/envs/pyscenic2/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [263]:
for i in range(100):
    rs=ps021[i].split(':')[1]
    rsl=int(rs.split('_')[0])
    rsr=int(rs.split('_')[1])
    rss=set(range(rsl,rsr))
    df221['ifr']=df221.r.apply(lambda x : len(rss.intersection(x)))
    pos=df221[df221['ifr']>0]
    if len(pos):
        print(pos['ifr'].values)
    else:
        print('########')